In [27]:
import xgboost as xgb
import numpy as np

In [19]:
# 1、xgBoost的基本使用
# 2、自定义损失函数的梯度和二阶导
# 3、binary:logistic/logitraw


# 定义f: theta * x
def log_reg(y_hat, y):
    p = 1.0 / (1.0 + np.exp(-y_hat))
    g = p - y.get_label()
    h = p * (1.0-p)
    return g, h


def error_rate(y_hat, y):
    return 'error', float(sum(y.get_label() != (y_hat > 0.5))) / len(y_hat)

In [28]:
# 读取数据
data_train = xgb.DMatrix('14.agaricus_train.txt')
data_test = xgb.DMatrix('14.agaricus_test.txt')
print data_train
print type(data_train)

<class 'xgboost.core.DMatrix'>


In [30]:
# 设置参数
param = {'max_depth': 2, 'eta': 1, 'silent': 0, 'objective': 'binary:logistic'} # logitraw
# param = {'max_depth': 3, 'eta': 0.3, 'silent': 1, 'objective': 'reg:logistic'}
watchlist = [(data_test, 'test'), (data_train, 'train')]
n_round = 5
bst = xgb.train(param, data_train, num_boost_round=n_round, evals=watchlist)
#bst = xgb.train(param, data_train, num_boost_round=n_round, evals=watchlist, obj=log_reg, feval=error_rate)

# 计算错误率
y_hat = bst.predict(data_test)
y = data_test.get_label()
print y_hat
print y

[0]	test-error:0.042831	train-error:0.046522
[1]	test-error:0.021726	train-error:0.022263
[2]	test-error:0.006207	train-error:0.007063
[3]	test-error:0.018001	train-error:0.0152
[4]	test-error:0.006207	train-error:0.007063
[ 0.08073306  0.92217326  0.08073306 ...,  0.98059034  0.01182149
  0.98059034]
[ 0.  1.  0. ...,  1.  0.  1.]


In [33]:
# 设置参数
param = {'max_depth': 4, 'eta': 1, 'silent': 0, 'objective': 'binary:logistic'} # logitraw
# param = {'max_depth': 3, 'eta': 0.3, 'silent': 1, 'objective': 'reg:logistic'}
watchlist = [(data_test, 'test'), (data_train, 'train')]
n_round = 5

#bst = xgb.train(param, data_train, num_boost_round=n_round, evals=watchlist)
bst = xgb.train(param, data_train, num_boost_round=n_round, evals=watchlist, obj=log_reg, feval=error_rate)

# 计算错误率
y_hat = bst.predict(data_test)
y = data_test.get_label()
print y_hat
print y

[0]	test-error:0.007449	train-error:0.006756	test-error:0.007449	train-error:0.006756
[1]	test-error:0	train-error:0.001228	test-error:0	train-error:0.001228
[2]	test-error:0	train-error:0.001228	test-error:0	train-error:0.001228
[3]	test-error:0	train-error:0.001228	test-error:0	train-error:0.001228
[4]	test-error:0	train-error:0.001228	test-error:0	train-error:0.001228
[  2.16791759e-05   9.99075532e-01   2.16791759e-05 ...,   9.99183118e-01
   2.34865856e-05   9.99183118e-01]
[ 0.  1.  0. ...,  1.  0.  1.]


In [34]:
error = sum(y != (y_hat > 0.5))
error_rate = float(error) / len(y_hat)
print '样本总数：\t', len(y_hat)
print '错误数目：\t%4d' % error
print '错误率：\t%.5f%%' % (100*error_rate)

样本总数：	1611
错误数目：	   0
错误率：	0.00000%


In [35]:
from sklearn.model_selection import train_test_split   # cross_validation


def iris_type(s):
    it = {'Iris-setosa': 0, 'Iris-versicolor': 1, 'Iris-virginica': 2}
    return it[s]


if __name__ == "__main__":
    path = u'..\\10.Regression\\10.iris.data'  # 数据文件路径
    data = np.loadtxt(path, dtype=float, delimiter=',', converters={4: iris_type})
    x, y = np.split(data, (4,), axis=1)
    x_train, x_test, y_train, y_test = train_test_split(x, y, random_state=1, test_size=50)

    data_train = xgb.DMatrix(x_train, label=y_train)
    data_test = xgb.DMatrix(x_test, label=y_test)
    watch_list = [(data_test, 'test'), (data_train, 'train')]
    param = {'max_depth': 2, 'eta': 1, 'silent': 0, 'obj': 'multi:softmax', 'num_class': 3}

    bst = xgb.train(param, data_train, num_boost_round=6, evals=watch_list)
    y_hat = bst.predict(data_test)
    result = y_test.reshape(1, -1) == y_hat
    print '正确率:\t', float(np.sum(result)) / len(y_hat)


[0]	test-merror:0.04	train-merror:0.04
[1]	test-merror:0.02	train-merror:0.02
[2]	test-merror:0.04	train-merror:0.01
[3]	test-merror:0.04	train-merror:0.01
[4]	test-merror:0.04	train-merror:0.01
[5]	test-merror:0.04	train-merror:0
正确率:	0.96


In [51]:
import xgboost as xgb
import numpy as np
from sklearn.model_selection import train_test_split   # cross_validation
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler


def show_accuracy(a, b, tip):
    acc = a.ravel() == b.ravel()
    print acc
    print tip + '正确率：\t', float(acc.sum()) / a.size


if __name__ == "__main__":
    data = np.loadtxt('14.wine.data', dtype=float, delimiter=',')
    y, x = np.split(data, (1,), axis=1)
    # x = StandardScaler().fit_transform(x)
    x_train, x_test, y_train, y_test = train_test_split(x, y, random_state=1, test_size=0.5)

    # Logistic回归
    lr = LogisticRegression(penalty='l2')
    lr.fit(x_train, y_train.ravel())
    y_hat = lr.predict(x_test)
    show_accuracy(y_hat, y_test, 'Logistic回归 ')


[ True  True  True  True  True  True  True  True  True  True  True False
  True  True  True  True  True  True  True  True  True  True False  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True False  True
  True  True  True  True  True  True  True False  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True False  True  True  True  True
  True  True  True  True  True]
Logistic回归 正确率：	0.943820224719


In [59]:
# XGBoost
y_train[y_train == 3] = 0
y_test[y_test == 3] = 0
data_train = xgb.DMatrix(x_train, label=y_train)
data_test = xgb.DMatrix(x_test, label=y_test)
watch_list = [(data_test, 'test'), (data_train, 'train')]
param = {'max_depth': 2, 'eta': 0.1, 'silent': 0, 'objective': 'multi:softmax', 'num_class': 3}
bst = xgb.train(param, data_train, num_boost_round=7, evals=watch_list)
y_hat = bst.predict(data_test)

[0]	test-merror:0.022472	train-merror:0.022472
[1]	test-merror:0.022472	train-merror:0.022472
[2]	test-merror:0.022472	train-merror:0.022472
[3]	test-merror:0.022472	train-merror:0.022472
[4]	test-merror:0.022472	train-merror:0.011236
[5]	test-merror:0.011236	train-merror:0.011236
[6]	test-merror:0	train-merror:0


In [60]:
show_accuracy(y_hat, y_test, 'XGBoost ')

[ True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True]
XGBoost 正确率：	1.0
